In [57]:
import pandas as pd

In [58]:
df = pd.read_csv("https://veribilimi.co/data/adult_dataset/adult.data")

In [59]:
# Veri seti hakkında detaylı bilgi: 
# https://www.rdocumentation.org/packages/arules/versions/1.6-3/topics/Adult
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# Hedef Değişken ve İncelenmesi

In [60]:
df.groupby(["output"]).count()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
output,,,,,,,,,,,,,,
<=50K,24720,24720,24720,24720,24720,24720,24720,24720,24720,24720,24720,24720,24720,24720
>50K,7841,7841,7841,7841,7841,7841,7841,7841,7841,7841,7841,7841,7841,7841


In [61]:
24720/7841

3.152659099604642

# Veri Keşfi 

In [62]:
df.describe()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
output            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


# Kategorik değişkenlerin groupby ile incelenmesi

### workclass

In [64]:
df.groupby(['workclass']).age \
.agg(['count']) \
.head(20)

,count
workclass,
?,1836
Federal-gov,960
Local-gov,2093
Never-worked,7
Private,22696
Self-emp-inc,1116
Self-emp-not-inc,2541
State-gov,1298
Without-pay,14


In [65]:
# Yukarıda ? işareti var acaba bu niye var?

In [66]:
df[df.workclass.str.contains("\\?", na=False)]\
.groupby(["output"]).age \
.agg(['count']) \
.head(30)
# df[df.Description.str.contains("COFFEE", na=False)].head()

,count
output,
<=50K,1645
>50K,191


In [67]:
1645/191

8.612565445026178

In [68]:
# Tüm veri içinde workclass için ? işareti output'a göre 3.15 oranında 
# dağılırken workclass'ın ? işareti içerdiğikayıtlarda bu oran 8.6'ya 
# çıkmış. Demekki 50K'dan küçük gelir gurubunda workclass ? oranı çok daha
# fazla.

In [69]:
df[ ( df.output.str.contains("\\<=", na=False)   )] \
.groupby(['workclass']).age \
.agg(['count']) \
.sort_values('count', ascending=False) \
.head()

,count
workclass,
Private,17733
Self-emp-not-inc,1817
?,1645
Local-gov,1476
State-gov,945


In [70]:
# 24720 50K içinde en çok Private var.

In [71]:
17733 / 24720

0.7173543689320389

In [72]:
# workclass içindeki ? işaretlerini doldurma konusundaki kararımız bunlar işsiz
# veya mesleği tanımlanamayan dolayısıyla biz de bunlara Unknown diyelim.

In [73]:
# work class içinde ? işareti olanları Unknown ile güncelleme
df.loc[df['workclass'].str.contains("\\?"), 'workclass'] = "Unknown"

In [75]:
# Sonuçları kontrol. Sağlıklı bir güncelleme olmuş mu?
df[ ( df.output.str.contains("\\<=", na=False)   )] \
.groupby(['workclass']).age \
.agg(['count']) \
.sort_values('count', ascending=False) \
.head()

,count
workclass,
Private,17733
Self-emp-not-inc,1817
Unknown,1645
Local-gov,1476
State-gov,945


# education kategorik değişkeninin incelenmesi

In [77]:
df.groupby(['education']).age \
.agg(['count']) \
.sort_values('count', ascending=False) \
.head(20)

,count
education,
HS-grad,10501
Some-college,7291
Bachelors,5355
Masters,1723
Assoc-voc,1382
11th,1175
Assoc-acdm,1067
10th,933
7th-8th,646


In [78]:
# Preschool < 1st-4th < 5th-6th < 7th-8th < 9th < 10th < 11th < 12th < HS-grad < Prof-school < Assoc-acdm < Assoc-voc < Some-college < Bachelors < Masters < Doctorate.

# Preschool:
# Elementaryschool: 1st-4th, 5th-6th, 7th-8th
# Highschool: 9th, 10th, 11th, 12th
# College: HS-grad, Prof-school, Assoc-acdm, Assoc-voc, Some-college 
# Bachelors:
# Masters:
# Doctorate:

In [82]:
# education2
education2 = []
for i in df.education:
    if i in ("Preschool"):
        education2.append("Preschool")
    elif i in ("1st-4th", "5th-6th", "7th-8th"):
        education2.append("Elementaryschool")
    elif i in ("9th", "10th", "11th", "12th"):
        education2.append("Highschool")
    elif i in ("HS-grad", "Prof-school", "Assoc-acdm", "Assoc-voc", "Some-college"):
        education2.append("College")
    elif i in ("Bachelors"):
        education2.append("Bachelors")
    elif i in ("Masters"):
        education2.append("Masters")
    elif i in ("Doctorate"):
        education2.append("Doctorate")
    else:education2.append("Unknown")
    

In [117]:
if "1st-4th" in ("1st-4th", "5th-6th", "7th-8th"):
    print(True)

True


In [83]:
df['education2'] = education2

In [84]:
df[['education','education2']].head(30)

,education,education2
0,Bachelors,Unknown
1,Bachelors,Unknown
2,HS-grad,Unknown
3,11th,Unknown
4,Bachelors,Unknown
5,Masters,Unknown
6,9th,Unknown
7,HS-grad,Unknown
8,Masters,Unknown
9,Bachelors,Unknown


In [94]:
# Preschool:
# Elementaryschool: 1st-4th, 5th-6th, 7th-8th
# Highschool: 9th, 10th, 11th, 12th
# College: HS-grad, Prof-school, Assoc-acdm, Assoc-voc, Some-college 
# Bachelors:
# Masters:
# Doctorate:
def change_education(i):
    my_dict = {
        "HS-grad":"College",
        "Some-college":"College",
        "Bachelors":"Bachelors",
        "Masters":"Masters",
        "Assoc-voc":"College",
        "11th":"Highschool",
        "Assoc-acdm":"College",
        "10th":"Highschool",
        "7th-8th":"Elementaryschool",
        "Prof-school":"College",
        "9th":"Highschool",
        "12th":"Highschool",
        "Doctorate":"Doctorate",
        "5th-6th":"Elementaryschool",
        "1st-4th":"Elementaryschool",
        "Preschool":"Preschool"     
    }
    
    return my_dict.get(i)

In [106]:
change_education("Some-college")

'College'

In [122]:
education2 = []
for i in df.education:
    education2.append(change_education(i))

In [120]:
df['education2'] = education2

In [121]:
df[['education','education2']].head()

,education,education2
0,Bachelors,None
1,Bachelors,None
2,HS-grad,None
3,11th,None
4,Bachelors,None
